## ASSIGNMENT 2 Task 1

### Data Reading,Exploration and preprocessing

In [28]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms,models
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((256,256)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

path = '/Users/dayveed/VscodeProjects/ML_labs/Assignment_2/data/'
train_dataset = datasets.Flowers102(root=path,split='train', transform=transform, download=True)
test_dataset = datasets.Flowers102(root=path,split='test', transform=transform, download=True)
val_dataset = datasets.Flowers102(root=path,split='val', transform=transform, download=True)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Machine learning or Deep learning model defining

In [29]:
# Define the baseline CNN model
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.features = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      nn.BatchNorm2d(32),
                                      nn.Dropout(p=0.3),

                                      nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      nn.BatchNorm2d(64),
                                      nn.Dropout(p=0.3),

                                      nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      nn.BatchNorm2d(128),
                                      nn.Dropout(p=0.3)
                                      )
        self.fc1 = nn.Sequential(nn.Linear(32 * 32 * 128, 512),
                                 nn.ReLU()
                                 )

        self.output = nn.Linear(512, 102)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.output(x)
        x = F.softmax(x, dim=1)
        # x = x.view(x.size(0), -1)  # Flatten
        # x = self.classifier(x)
        return x

BaselineModel()

BaselineModel(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.3, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.3, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout(p=0.3, inplace=False)
  )
  (fc1): Sequential(
    (0): Linear(in_

#### Specify loss func and optimizer for base model

In [30]:
# Initialize the model, optimizer, and loss function
device = (
    'cpu'
    # "mps" 
    # if torch.backends.mps.is_available() 
    # else "cuda" 
    # if torch.cuda.is_available() 
    # else "cpu"
)

print(f"Using device: {device}")
model = BaselineModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

Using device: cpu


### Training model func

In [31]:
# ## YOUR CODE HERE ##
# num_epochs = 10

# # Establish a list for our history
# train_loss_history = list()
# val_loss_history = list()

# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0.0
#     train_correct = 0
#     for i, data in enumerate(train_loader):
#         # data is a list of [inputs, labels]
#         inputs, labels = data

#         # Pass to GPU if available.
#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()

#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         _, preds = torch.max(outputs.data, 1)
#         train_correct += (preds == labels).sum().item()
#         train_loss += loss.item()
#     print(f'Epoch {epoch + 1} training accuracy: {train_correct/len(train_loader.dataset):.2f}% training loss: {train_loss/len(train_loader.dataset):.5f}')
#     train_loss_history.append(train_loss/len(train_loader.dataset))


#     val_loss = 0.0
#     val_correct = 0
#     model.eval()
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)

#             outputs = model(inputs)
#             loss = criterion(outputs, labels)

#             _, preds = torch.max(outputs.data, 1)
#             val_correct += (preds == labels).sum().item()
#             val_loss += loss.item()
#         print(f'Epoch {epoch + 1} validation accuracy: {val_correct/len(val_loader.dataset):.4f}% validation loss: {val_loss/len(val_loader.dataset):.5f}')
#         val_loss_history.append(val_loss/len(val_loader.dataset))




def training_model(writer:SummaryWriter,criterion,optimizer,model,epochs:int = 10):
    # Initialize TensorBoard writer
    writer = SummaryWriter()
    epochs = 10
    # Train the model
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        for i, (images, labels) in enumerate(train_loader):
            # Move data to device
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Update training loss and accuracy
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
        
        # Calculate training loss and accuracy
        train_loss /= len(train_loader)
        train_accuracy = train_correct / len(train_dataset)
        
        # Evaluate the model on the val set
        model.eval()
        val_loss = 0
        val_correct = 0
        val_labels = []
        val_predicted = []
        with torch.no_grad():
            for images, labels in val_loader:
                # Move data to device
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_labels.extend(labels.cpu().numpy())
                val_predicted.extend(predicted.cpu().numpy())
        
        # Calculate test loss and accuracy
        val_loss /= len(val_loader)
        val_accuracy = val_correct / len(val_dataset)
        
        # Calculate F1 score
        test_f1 = f1_score(val_labels, val_predicted, average='macro')
        
        # Log metrics to TensorBoard
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_accuracy, epoch)
        writer.add_scalar('Loss/Val', val_loss, epoch)
        writer.add_scalar('Accuracy/Val', val_accuracy, epoch)
        writer.add_scalar('F1/Val', test_f1, epoch)
        
        # Print metrics
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Epoch {epoch+1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {test_f1:.4f}')

    # Close TensorBoard writer
    writer.close()

#### Train base model

In [32]:
# training_model(writer=SummaryWriter(),criterion=criterion,optimizer=optimizer,model=model)

## Transfer Learning using VGG

#### Define VGG model

In [ ]:
vgg_model = models.vgg16(pretrained=True)
for param in vgg_model.parameters():
    param.requires_grad = False
    
vgg_model.classifier = nn.Sequential(
                        nn.Linear(25088, 512), 
                        nn.ReLU(),
                        nn.Dropout(p=0.2),
    
                        nn.Linear(512, 256),
                        nn.ReLU(),
                        nn.Dropout(p=0.2),
                                 
                        nn.Linear(256, 102),
                        nn.LogSoftmax(dim=1))

vgg_model = vgg_model.to(device)

# Define the loss
# criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(vgg_model.classifier.parameters(), lr=0.001)

vgg_model

/Users/dayveed/VscodeProjects/ML_labs/.env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dayveed/VscodeProjects/ML_labs/.env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

#### Train VGG

In [34]:
training_model(writer=SummaryWriter(),criterion=criterion,optimizer=optimizer,model=vgg_model)

Epoch 1, Train Loss: 4.2895, Train Accuracy: 0.0863
Epoch 1, Val Loss: 3.2234, Val Accuracy: 0.3039, Val F1: 0.2568
Epoch 2, Train Loss: 2.1527, Train Accuracy: 0.4657
Epoch 2, Val Loss: 1.7392, Val Accuracy: 0.5814, Val F1: 0.5410
Epoch 3, Train Loss: 0.7793, Train Accuracy: 0.8020
Epoch 3, Val Loss: 1.3422, Val Accuracy: 0.6500, Val F1: 0.6353
Epoch 4, Train Loss: 0.2902, Train Accuracy: 0.9255
Epoch 4, Val Loss: 1.1669, Val Accuracy: 0.7020, Val F1: 0.6999
Epoch 5, Train Loss: 0.1422, Train Accuracy: 0.9657
Epoch 5, Val Loss: 1.2175, Val Accuracy: 0.6951, Val F1: 0.6997
Epoch 6, Train Loss: 0.0789, Train Accuracy: 0.9843
Epoch 6, Val Loss: 1.0419, Val Accuracy: 0.7147, Val F1: 0.7112
Epoch 7, Train Loss: 0.0532, Train Accuracy: 0.9863
Epoch 7, Val Loss: 1.1299, Val Accuracy: 0.7049, Val F1: 0.6957
Epoch 8, Train Loss: 0.0562, Train Accuracy: 0.9882
Epoch 8, Val Loss: 1.0841, Val Accuracy: 0.7157, Val F1: 0.7114
Epoch 9, Train Loss: 0.0676, Train Accuracy: 0.9814
Epoch 9, Val Loss: 1

## Transfer Learning using ResNet-50

#### Define ResNet-50 model

In [35]:
resnet_model = models.resnet50()
for param in resnet_model.parameters():
    param.requires_grad = False
    
resnet_model.classifier = nn.Sequential(
                        nn.Linear(25088, 512), 
                        nn.ReLU(),
                        nn.Dropout(p=0.2),
    
                        nn.Linear(512, 256),
                        nn.ReLU(),
                        nn.Dropout(p=0.2),
                                 
                        nn.Linear(256, 102),
                        nn.LogSoftmax(dim=1))

resnet_model = resnet_model.to(device)
# Define the optimizer
optimizer = optim.Adam(resnet_model.classifier.parameters(), lr=0.001)

resnet_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

#### Train ResNet-50 model

In [36]:
training_model(writer=SummaryWriter(),criterion=criterion,optimizer=optimizer,model=vgg_model)

Epoch 1, Train Loss: 0.0378, Train Accuracy: 0.9931
Epoch 1, Val Loss: 1.1312, Val Accuracy: 0.7088, Val F1: 0.6998


KeyboardInterrupt: 

### Model performance evaluation

### Conclusion and possible improvements